In [1]:
import time, re, json, os
from datetime import date
today = date.today()
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import numpy as np
import pandas as pd
from googlesearch import search 

In [2]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

namus = pd.read_csv("namus.csv")
print(namus)

       Unnamed: 0 Case Number         DLC  Last Name First Name Missing Age  \
0               1     MP73132  08/23/2020    Ragland    Kristie    52 Years   
1               2     MP67982  03/08/2020     Watson    Douglas    29 Years   
2               3     MP64666  01/16/2020    Edwards    Katrina    16 Years   
3               4     MP67978  01/08/2020  Kennebrew       Judy    65 Years   
4               5     MP71439  10/21/2019     Taylor     Lonnie    78 Years   
...           ...         ...         ...        ...        ...         ...   
21000          43      MP6400  08/04/1974      Meyer    Deborah    15 Years   
21001          44     MP14488  07/04/1974      Brown    Carlene    19 Years   
21002          45     MP13812  04/27/1974     Morris      Larry    24 Years   
21003          46     MP23115  07/12/1966    Johnson     Dennis     8 Years   
21004          47     MP27783  09/17/1934     Mauger       Olga    21 Years   

                            City      County State 

In [3]:
#google query
query = "\"" + str(namus.iloc[0,3]) + " " + str(namus.iloc[0,4]) + "\""
print(query)
for j in search(query, tld="co.in", num=10, stop=10, pause=2): 
    print(j) 

"Ragland Kristie"
https://www.beenverified.com/people/kristie-ragland/
https://www.facebook.com/public/Kirstie-Ragud
https://www.rollerfuneralhomes.com/services.asp?locid=&page=odetail&id=39539
https://www.coffmanfh.com/obituary/3666208
https://www.legacy.com/obituaries/name/alice-ragland-obituary?pid=16058765
https://www.legacy.com/obituaries/dailyhome/obituary.aspx?n=don-lee-byers-pockeye&pid=186536458&fhid=5377
https://clustrmaps.com/person/Ragland-2i6jgo
https://www.findagrave.com/memorial/29106447/bennie-mae-smith
https://www.spokeo.com/Kristie-Marsh
https://www.whitepages.com/name/Kristie-Ragland


In [21]:
default_download_dir = "./PACER_data"
os.system("mkdir -p " + default_download_dir)

options = Options()
options.add_experimental_option("prefs", {
  "download.default_directory": "./PACER_data",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)

browser.get("https://pcl.uscourts.gov/pcl/index.jsf")
#then manually enter login information

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [/Users/flynn_chen/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [22]:
for i in range(len(namus)):
    
    if (i == 2):
        break

    browser.get("https://pcl.uscourts.gov/pcl/pages/search/findPartyAdvanced.jsf")
    time.sleep(3)

    # input: 
    last_name = namus["Last Name"][i]
    first_name = namus["First Name"][i]
    state = abbrev_us_state[namus["State"][i]]

    last_name_entry = browser.find_elements_by_xpath('''//*[@id="frmSearch:txtPartyNameLast"]''')[0]
    last_name_entry.click()
    last_name_entry.send_keys(last_name)

    time.sleep(0.5)

    first_name_entry = browser.find_elements_by_xpath('''//*[@id="frmSearch:txtPartyNameFirst"]''')[0]
    first_name_entry.click()
    first_name_entry.send_keys(first_name)

    time.sleep(0.5)

    exact_match = browser.find_elements_by_xpath('''//*[@id="frmSearch:cbExactMatches"]/div[2]/span''')[0]
    exact_match.click()

    time.sleep(0.5)

    region_entry = browser.find_elements_by_xpath('''//*[@id="frmSearch:scmRegion"]/ul''')[0]
    region_entry.click()

    time.sleep(0.5)

    region_entry = browser.find_elements_by_xpath('''//*[@id="frmSearch:scmRegion_panel"]/div[1]/div[2]/input''')[0]
    region_entry.click()
    region_entry.send_keys("federal")

    time.sleep(1)

    region_check = browser.find_elements_by_xpath('''//*[@id="frmSearch:scmRegion_panel"]/div[1]/div[1]/div[2]/span''')[0]
    region_check.click()

    time.sleep(0.5)

    region_entry.clear()
    region_entry.send_keys(state)
    region_check.click()
    
    time.sleep(0.5)

    search_buttom = browser.find_elements_by_xpath('''//*[@id="frmSearch:btnSearch"]/span''')[0]
    search_buttom.click()
    
    time.sleep(2)
    
    #if there are results
    results_count = browser.find_elements_by_xpath('''//*[@id="pnlResultCount"]/span[1]''')
    if len(results_count) == 0:
        print("There are no results for " + first_name + " " + last_name)
        no_result_ok = browser.find_elements_by_xpath('''//*[@id="frmSearch:btnNoResultsOkay"]''')[0]
        no_result_ok.click()
        continue
    
    num_results = int(results_count[0].text)
    print("There are " + str(num_results) + " results for " + first_name + " " + last_name)
    if num_results > 0:
        
        download_button = browser.find_elements_by_xpath('''//*[@id="btnDownloadResults"]''')[0]
        download_button.click()
        time.sleep(1)
        
        csv_link = browser.find_elements_by_xpath(''' //*[@id="frmDownloadResults:opt3"]/div[2]/span''')[0]
        csv_link.click()
        time.sleep(0.5)
        
        download_link = browser.find_elements_by_xpath('''//*[@id="frmDownloadResults:btnBeginDownload"]''')[0]
        download_link.click()
        time.sleep(0.5)
        
        pacer_download_file_regex = today.strftime("-%m.%d.%Y-")
        os.system("mv ~/Downloads/*" + pacer_download_file_regex + "*.csv " + 
                  default_download_dir + "/" + last_name + "_" + first_name + "." + namus["State"][i] + ".PACER.csv")
        
        continue


There are no results for Kristie Ragland
There are 9 results for Douglas Watson


In [ ]:
browser.close()